# Distributions

This tutorial demonstrates the functionality of the distributions module. It covers operations like sampling from distributions, calculating log probabilities, and checking the number of parameters.

## Libraries

To get started, you'll need to import some essential libraries. The specific libraries you use will depend on the backend you've chosen, such as PyTorch, TensorFlow, or Jax. Additionally, you'll need to import NumPy.

In [ ]:
import torch
import numpy as np
import tensorflow as tf

## Functions

The ``compare_tensors`` function checks whether two tensors are close in value, given a relative and absolute tolerance. It prints the maximum absolute difference between the tensors and returns whether they are close.

In [ ]:
def compare_tensors(a, b, rtol=1e-1, atol=1e-1, name=""):
    are_close = np.allclose(a, b, rtol=rtol, atol=atol)
    max_diff = np.max(np.abs(a - b))
    print(f"{name} are close: {are_close}")
    print(f"Max absolute difference for {name}: {max_diff}")
    return are_close

## Random seeds

Set random seeds for reproducibility. This ensures that the results are consistent each time the code is executed.

In [ ]:
np.random.seed(0)
torch.manual_seed(0)
tf.random.set_seed(0)

## Illia

When setting the backend, we import the Illia library, which provides Bayesian module implementations. Note that backend selection requires a kernel restart and cannot be changed dynamically.

In [ ]:
import illia

# Display available backends
illia.show_available_backends()

## Test parameters and utilities

Define parameters and utilities that will be used in the tests.

In [ ]:
shape = (3, 2)  # Shape of the distribution
mu_init = 0.0  # Initial mean
rho_init = -7.0  # Initial rho

## Initialize class distributions

Import and initialize Gaussian distribution classes for the various frameworks.

In [ ]:
from illia.torch.distributions.dynamic.gaussian import (
    GaussianDistribution as TorchGaussianDistribution,
)
from illia.tf.distributions.dynamic.gaussian import (
    GaussianDistribution as TFGaussianDistribution,
)

# PyTorch
torch_dynamic_dist = TorchGaussianDistribution(
    shape=shape, mu_init=mu_init, rho_init=rho_init
)

# Tensorflow
tf_dynamic_dist = TFGaussianDistribution(
    shape=shape, mu_init=mu_init, rho_init=rho_init
)

## Sampling

Sample from the distributions and compare the means and standard deviations.

In [ ]:
print("Test 1: Sampling")

n_samples = 10000  # Number of samples
torch_samples = np.array(
    [torch_dynamic_dist.sample().detach().cpu().numpy() for _ in range(n_samples)]
)
tf_samples = np.array([tf_dynamic_dist.sample().numpy() for _ in range(n_samples)])

# Compare means
torch_mean = np.mean(torch_samples, axis=0)
tf_mean = np.mean(tf_samples, axis=0)
compare_tensors(torch_mean, tf_mean, name="Means")

# Compare standard deviations
torch_std = np.std(torch_samples, axis=0)
tf_std = np.std(tf_samples, axis=0)
compare_tensors(torch_std, tf_std, name="Standard deviations")

## Log probabilities

Calculate and compare the log probabilities of a sample for both distributions.

In [ ]:
print("\nTest 2: Log probability")

x = np.random.randn(*shape).astype(np.float32)
torch_log_prob = (
    torch_dynamic_dist.log_prob(torch.tensor(x, dtype=torch.float32))
    .detach()
    .cpu()
    .numpy()
)
tf_log_prob = tf_dynamic_dist.log_prob(tf.constant(x, dtype=tf.float32)).numpy()
compare_tensors(
    torch_log_prob, tf_log_prob, rtol=1e-1, atol=1e-1, name="Log probabilities"
)

## Number of parameters

Check the number of parameters in both distributions to ensure they match.

In [ ]:
print("\nTest 3: Number of parameters")
print("PyTorch num params:", torch_dynamic_dist.num_params)
print("TensorFlow num params:", tf_dynamic_dist.num_params)
print(
    "Num params are equal:", torch_dynamic_dist.num_params == tf_dynamic_dist.num_params
)